# Driver Code

In [33]:
from sklearn.model_selection import train_test_split

class Experiment:
    def __init__(self, X, y, train_size=0.8, test_size=0.1, val_size=0.1, split_type='train-val-test', print_stats=None):
        self.X = X
        self.y = y

        self.X_train = None
        self.y_train = None
        self.X_val = None
        self.y_val = None
        self.X_test = None
        self.y_test = None

        self.__split_data(train_size, test_size, val_size, split_type)
        if print_stats:
            self.__print_data_summary(train_size, test_size, val_size, split_type)

    def __split_data(self, train_size, test_size, val_size, split_type):
        if split_type == 'train-val-test':
            if not np.isclose(train_size + test_size + val_size, 1.0):
                raise ValueError("train_size, test_size, and val_size must sum to 1.0")

            if train_size == 1.0:
                self.X_train, self.y_train = self.X, self.y
                return

            X_train, X_temp, y_train, y_temp = train_test_split(
                self.X, self.y, train_size=train_size, random_state=42
            )

            self.X_train, self.y_train = X_train, y_train

            remaining_size = val_size + test_size
            if np.isclose(remaining_size, 0.0):
                return

            relative_test_size = test_size / remaining_size

            if np.isclose(relative_test_size, 1.0):
                self.X_test, self.y_test = X_temp, y_temp
            elif np.isclose(relative_test_size, 0.0):
                self.X_val, self.y_val = X_temp, y_temp
            else:
                self.X_val, self.X_test, self.y_val, self.y_test = train_test_split(
                    X_temp, y_temp, test_size=relative_test_size, random_state=42
                )

        elif split_type == 'train-test':
            if not np.isclose(train_size + test_size, 1.0):
                raise ValueError("train_size and test_size must sum to 1.0")

            if train_size == 1.0:
                self.X_train, self.y_train = self.X, self.y
            elif test_size == 1.0:
                self.X_test, self.y_test = self.X, self.y
            else:
                self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                    self.X, self.y, train_size=train_size, random_state=42
                )

        else:
            raise ValueError(f"Unknown split_type: {split_type}. Must be 'train-val-test' or 'train-test'.")

    def __print_data_summary(self, train_size, test_size, val_size, split_type):
        total_samples = len(self.X)
        train_samples = len(self.X_train) if self.X_train is not None else 0
        val_samples = len(self.X_val) if self.X_val is not None else 0
        test_samples = len(self.X_test) if self.X_test is not None else 0

        print(f"\n--- Experiment Data Initialized ---")
        print(f"Total Samples: {total_samples}")
        print(f"Split Type:    '{split_type}'")

        if split_type == 'train-val-test':
            print(f"  - Train: {train_size*100:>6.1f}% ({train_samples} samples)")
            print(f"  - Val:   {val_size*100:>6.1f}% ({val_samples} samples)")
            print(f"  - Test:  {test_size*100:>6.1f}% ({test_samples} samples)")
        elif split_type == 'train-test':
            print(f"  - Train: {train_size*100:>6.1f}% ({train_samples} samples)")
            print(f"  - Test:  {test_size*100:>6.1f}% ({test_samples} samples)")

        print(f"Total in splits: {train_samples + val_samples + test_samples}")
        print("-----------------------------------\n")


In [47]:
import time
from scipy.spatial.distance import cdist
from cvxopt import matrix, solvers

from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path

solvers.options['show_progress'] = False
OUTPUT_PATH = Path("results_qsvr")


class QuantileSVRExperiment(Experiment):
    def __init__(self, X, y, satellite, train_size=0.8, test_size=0.1, val_size=0.1,
                 split_type='train-val-test', print_stats=None, type='censored'):
        super().__init__(X, y, train_size, test_size, val_size, split_type, print_stats)
        self.satellite = satellite
        self.results_path = OUTPUT_PATH / f"qsvr_pi_estimation_{type}"
        os.makedirs(self.results_path, exist_ok=True)

        self.__scale_data()

    def __scale_data(self):
        """Scales X and Y data, storing the scalers."""
        self.x_scaler = MinMaxScaler()
        self.y_scaler = MinMaxScaler()


        self.X_train_scaled = self.x_scaler.fit_transform(self.X_train)
        self.X_val_scaled = self.x_scaler.transform(self.X_val)
        self.X_test_scaled = self.x_scaler.transform(self.X_test)


        self.y_train = self.y_train.reshape(-1, 1)
        self.y_val = self.y_val.reshape(-1, 1)
        self.y_test = self.y_test.reshape(-1, 1)


        self.y_train_scaled = self.y_scaler.fit_transform(self.y_train)
        self.y_val_scaled = self.y_scaler.transform(self.y_val)
        self.y_test_scaled = self.y_scaler.transform(self.y_test)


        self.y_train = self.y_train.ravel()
        self.y_val = self.y_val.ravel()
        self.y_test = self.y_test.ravel()
        self.y_train_scaled = self.y_train_scaled.ravel()
        self.y_val_scaled = self.y_val_scaled.ravel()
        self.y_test_scaled = self.y_test_scaled.ravel()



    @staticmethod
    def kernelfun(X, kerfPara, Y=None):
        """Evaluate the RBF kernel."""
        if Y is None:
            Y = X
        if kerfPara['type'] == 'rbf':
            gamma = kerfPara['pars']
            sqdist = cdist(X, Y, 'sqeuclidean')
            return np.exp(-gamma * sqdist)
        else:
            raise ValueError("Unknown kernel function")

    @staticmethod
    def svtol(C):
        """A helper function to set a tolerance based on C."""
        return 1e-5

    @staticmethod
    def nobias(kerfType):
        """For this demonstration we simply disable the bias-computation by returning 0."""
        return 0

    @staticmethod
    def fit_quantile_svr(X, Y, kerfPara, C, tau, eps1=0.0):
        """
        Trains the Quantile SVR model by solving the Quadratic Program.

        Returns:
          beta (np.array): The learned model coefficients.
          bias (float): The learned model bias.
        """
        epsilon = QuantileSVRExperiment.svtol(C)
        n = X.shape[0]
        H = QuantileSVRExperiment.kernelfun(X, kerfPara)  # shape: (n,n)


        Hb_top = np.hstack([H, -H])
        Hb_bottom = np.hstack([-H, H])
        Hb = np.vstack([Hb_top, Hb_bottom])



        Y_col = Y.reshape(-1, 1)
        c_part1 = ((1 - tau) * eps1 * np.ones((n, 1)) - Y_col)
        c_part2 = (tau * eps1 * np.ones((n, 1)) + Y_col)
        c_vec = np.vstack([c_part1, c_part2]).flatten()


        vlb = np.zeros(2 * n)
        vub = np.concatenate([tau * C * np.ones(n), (1 - tau) * C * np.ones(n)])

        A = None
        b_eq = None



        P = matrix(Hb)
        q = matrix(c_vec)
        I = np.eye(2 * n)
        G1 = -I
        h1 = np.zeros(2 * n)
        G2 = I
        h2 = vub
        G = matrix(np.vstack([G1, G2]))
        h = matrix(np.hstack([h1, h2]))

        sol = solvers.qp(P, q, G, h)

        alpha = np.array(sol['x']).flatten()
        alpha1 = alpha[:n]
        beta1 = alpha[n:2*n]
        beta = alpha1 - beta1


        bias = 0

        return beta, bias, H

    @staticmethod
    def predict_quantile_svr(X_train, X_predict, kerfPara, beta, bias):
        """
        Generates predictions on new data using a trained Q-SVR model.
        """
        H_test = QuantileSVRExperiment.kernelfun(X_predict, kerfPara, X_train)
        PredictY = H_test.dot(beta) + bias
        return PredictY

    def evaluate_model(self, y_true, y_pred_lower, y_pred_upper):
        """
        Evaluates the prediction interval using PICP and MPIW.
        (Adapted from your PredictionIntervalEstimation class)
        """
        y_true_flat = y_true.flatten()
        y_lower_flat = y_pred_lower.flatten()
        y_upper_flat = y_pred_upper.flatten()

        def picp(y_true_vals, y_pred_lower_vals, y_pred_upper_vals):
            """Prediction Interval Coverage Probability"""
            covered = np.sum((y_true_vals >= y_pred_lower_vals) & (y_true_vals <= y_pred_upper_vals))
            return covered / len(y_true_vals)

        def mpiw(y_pred_lower_vals, y_pred_upper_vals):
            """Mean Prediction Interval Width"""
            return np.mean(y_pred_upper_vals - y_pred_lower_vals)

        return {
            'PICP': float(picp(y_true_flat, y_lower_flat, y_upper_flat)),
            'MPIW': float(mpiw(y_lower_flat, y_upper_flat))
        }

    def plot_prediction_interval(self, y_pred_lower_test, y_pred_upper_test,
                                 y_pred_lower_val, y_pred_upper_val, model_param_string):
        """
        Plots the prediction intervals for the test set.
        (Adapted from your PredictionIntervalEstimation class)
        """
        indices = range(len(self.y_test))

        test_eval_dict = self.evaluate_model(self.y_test, y_pred_lower_test, y_pred_upper_test)
        val_eval_dict = self.evaluate_model(self.y_val, y_pred_lower_val, y_pred_upper_val)

        plt.figure(figsize=(14, 7))
        plt.plot(indices, self.y_test, 'o', color='blue', label='Actual Soil Moisture (Test Set)', markersize=4)
        plt.plot(indices, y_pred_lower_test, color='red', linestyle='--', label='Lower Bound')
        plt.plot(indices, y_pred_upper_test, color='orange', linestyle='--', label='Upper Bound')

        plt.fill_between(indices, y_pred_lower_test, y_pred_upper_test, color='gray', alpha=0.2, label='95% Prediction Interval')

        test_metrics = f"Test  | PICP: {test_eval_dict['PICP']*100:5.2f}% | MPIW: {test_eval_dict['MPIW']:.4f}"
        val_metrics =  f"Valid | PICP: {val_eval_dict['PICP']*100:5.2f}% | MPIW: {val_eval_dict['MPIW']:.4f}"
        metrics_text = f"{test_metrics}\n{val_metrics}"

        plt.annotate(metrics_text, xy=(0.02, 0.98), xycoords='axes fraction',
                    bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.8),
                    verticalalignment='top', fontsize=12, fontname='monospace')

        plot_dir = self.results_path / "plots"
        os.makedirs(plot_dir, exist_ok=True)

        plt.xlabel('Sample Index')
        plt.ylabel('Soil Moisture')
        plt.title(f'{self.satellite}: {model_param_string}\nQ-SVR Prediction Interval')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plot_save_path = plot_dir / f"{self.satellite}_{model_param_string}.png"
        plt.savefig(plot_save_path, dpi=300)
        print(f"Plot saved to {plot_save_path}")
        plt.close()

    def run_experiment(self, C_value, gamma, q_lower=0.025, q_upper=0.975, eps1=0.0):
        """
        Runs the full experiment for training and evaluating the Q-SVR.
        """
        model_param_string = f"C={C_value}_gamma={gamma}"
        print(f"\n--- Running Quantile SVR for {self.satellite} ---")
        print(f"Params: {model_param_string}")


        kerfPara = {'type': 'rbf', 'pars': gamma}


        print(f"Training lower quantile ({q_lower}) model...")
        start_time = time.time()

        beta_lower, bias_lower, _ = self.fit_quantile_svr(
            self.X_train_scaled, self.y_train, kerfPara, C_value, q_lower, eps1
        )
        print(f"Done in {time.time() - start_time:.2f}s")


        print(f"Training upper quantile ({q_upper}) model...")
        start_time = time.time()
        beta_upper, bias_upper, _ = self.fit_quantile_svr(
            self.X_train_scaled, self.y_train, kerfPara, C_value, q_upper, eps1
        )
        print(f"Done in {time.time() - start_time:.2f}s")


        print("Generating predictions...")

        y_preds_lower_val = self.predict_quantile_svr(
            self.X_train_scaled, self.X_val_scaled, kerfPara, beta_lower, bias_lower
        )
        y_preds_upper_val = self.predict_quantile_svr(
            self.X_train_scaled, self.X_val_scaled, kerfPara, beta_upper, bias_upper
        )


        y_preds_lower_test = self.predict_quantile_svr(
            self.X_train_scaled, self.X_test_scaled, kerfPara, beta_lower, bias_lower
        )
        y_preds_upper_test = self.predict_quantile_svr(
            self.X_train_scaled, self.X_test_scaled, kerfPara, beta_upper, bias_upper
        )


        print("Evaluating and plotting results...")
        self.plot_prediction_interval(
            y_preds_lower_test, y_preds_upper_test,
            y_preds_lower_val, y_preds_upper_val,
            model_param_string
        )

        results_val = self.evaluate_model(self.y_val, y_preds_lower_val, y_preds_upper_val)
        results_test = self.evaluate_model(self.y_test, y_preds_lower_test, y_preds_upper_test)

        results = {
            "params": {"C": C_value, "gamma": gamma, "q_lower": q_lower, "q_upper": q_upper},
            "val": results_val,
            "test": results_test
        }

        print(f"Results for {model_param_string}:")
        print(json.dumps(results, indent=4))


        metrics_filename = self.results_path / f"{self.satellite}_metrics_{model_param_string}.json"
        # with open(metrics_filename, "w") as f:
        #     json.dump(results, f, indent=4)
        # print(f"Metrics saved to {metrics_filename}")

        return results

# Experiment Code

In [35]:
import pandas as pd

eos = pd.read_csv("data/eos-04-processed.csv")
sentinel = pd.read_csv("data/sentinel-1-processed.csv")

In [36]:
sentinel = sentinel[sentinel['SM1 (%)'] != 50]
eos = eos[eos['SM1 (%)'] != 50]

In [37]:
X_cols_eos = ['HH-pol', 'HV-pol']
X_cols_sentinel = ['VH-pol', 'VV-pol']

y_col = ['SM1 (%)']

X_sentinel = sentinel[X_cols_sentinel].values
X_eos = eos[X_cols_eos].values

y_sentinel = sentinel[y_col].values
y_eos = eos[y_col].values

In [ ]:
C_VALUE = 2**6
GAMMA_VALUES = [2 ** i for i in range(-15, 16)]

GAMMA_VALUES

In [41]:
sentinel_exp = QuantileSVRExperiment(
    X=X_sentinel,
    y=y_sentinel,
    satellite="Sentinel-1",
    print_stats=False,
    type="uncensored" # or "uncensored"
)

all_results = []

for i, gamma in enumerate(GAMMA_VALUES):
    print(f"\n[RUN {i+1}/{len(GAMMA_VALUES)}]")
    print(f"Testing Gamma = {gamma} (2^{np.log2(gamma):.0f})")

    try:
        results = sentinel_exp.run_experiment(
            C_value=C_VALUE,
            gamma=gamma
        )
        all_results.append(results)
        # print(f"Run {i+1} complete. Test PICP: {results['test']['PICP']:.4f}, Test MPIW: {results['test']['MPIW']:.4f}")

    except Exception as e:
        print(f"ERROR: Run failed for Gamma = {gamma}")
        print(f"Details: {e}")
        all_results.append({
            "params": {"C": C_VALUE, "gamma": gamma},
            "val": {"PICP": None, "MPIW": None},
            "test": {"PICP": None, "MPIW": None},
            "error": str(e)
        })


[RUN 1/31]
Testing Gamma = 3.0517578125e-05 (2^-15)

--- Running Quantile SVR for Sentinel-1 ---
Params: C=64_gamma=3.0517578125e-05
Training lower quantile (0.025) model...
Done in 27.45s
Training upper quantile (0.975) model...
Done in 28.73s
Generating predictions...
Evaluating and plotting results...
Plot saved to results_qsvr/qsvr_pi_estimation_uncensored/plots/Sentinel-1_C=64_gamma=3.0517578125e-05.png
Results for C=64_gamma=3.0517578125e-05:
{
    "params": {
        "C": 64,
        "gamma": 3.0517578125e-05,
        "q_lower": 0.025,
        "q_upper": 0.975
    },
    "val": {
        "PICP": 0.9605263157894737,
        "MPIW": 43.06355146256459
    },
    "test": {
        "PICP": 0.954248366013072,
        "MPIW": 43.063624304912594
    }
}
Metrics saved to results_qsvr/qsvr_pi_estimation_uncensored/Sentinel-1_metrics_C=64_gamma=3.0517578125e-05.json

[RUN 2/31]
Testing Gamma = 6.103515625e-05 (2^-14)

--- Running Quantile SVR for Sentinel-1 ---
Params: C=64_gamma=6.103515

In [43]:
results_df = pd.json_normalize(all_results)
results_df.columns = results_df.columns.str.replace('params.', 'param_')

try:
    # Calculate the exponent (e.g., 11 from 2048)
    gamma_exponents = np.log2(results_df['param_gamma']).astype(int)
    # Create the new string column (e.g., "2^11")
    results_df['param_gamma_str'] = '2^' + gamma_exponents.astype(str)
except Exception as e:
    print(f"Could not create 'param_gamma_str': {e}")
    results_df['param_gamma_str'] = results_df['param_gamma'] # Fallback
# --- END NEW CODE ---

# Save the full summary to a CSV
summary_filename = sentinel_exp.results_path / f"tuning_summary_C={C_VALUE}.csv"
results_df.to_csv(summary_filename, index=False)
print(f"\nFull tuning summary saved to:\n{summary_filename}")

# --- Analysis ---
acceptable_coverage_df = results_df[
    (results_df['test.PICP'] > 0.90) & (results_df['test.PICP'] <= 0.97)
].copy()

if not acceptable_coverage_df.empty:
    acceptable_coverage_df = acceptable_coverage_df.sort_values(by="test.MPIW", ascending=True)
    print("\n--- Top Models with 90-97% Test PICP (sorted by width) ---")
    print(acceptable_coverage_df[['param_gamma_str', 'test.PICP', 'test.MPIW']].head())
else:
    print("\nNo models achieved acceptable test PICP (90-97%).")

print("\n--- Top Models (sorted by Test PICP) ---")
print(results_df.sort_values(by="test.PICP", ascending=False)[['param_gamma_str', 'test.PICP', 'test.MPIW']].head(5))


Full tuning summary saved to:
results_qsvr/qsvr_pi_estimation_uncensored/tuning_summary_C=64.csv

--- Top Models with 90-97% Test PICP (sorted by width) ---
   param_gamma_str  test.PICP  test.MPIW
21             2^6   0.908497  34.208806
20             2^5   0.921569  35.677933
19             2^4   0.915033  35.807349
18             2^3   0.947712  36.458744
17             2^2   0.941176  37.252374

--- Top Models (sorted by Test PICP) ---
  param_gamma_str  test.PICP  test.MPIW
0           2^-15   0.954248  43.063624
1           2^-14   0.954248  43.060581
2           2^-13   0.954248  43.054495
3           2^-12   0.954248  43.042325
4           2^-11   0.954248  43.018478


In [46]:
results_df.sort_values(by="test.PICP", ascending=False)[['param_gamma_str', 'test.PICP', 'test.MPIW']]

,param_gamma_str,test.PICP,test.MPIW
0,2^-15,0.954248,43.063624
1,2^-14,0.954248,43.060581
2,2^-13,0.954248,43.054495
3,2^-12,0.954248,43.042325
4,2^-11,0.954248,43.018478
5,2^-10,0.954248,42.938109
6,2^-9,0.954248,42.852435
7,2^-8,0.954248,42.759538
10,2^-5,0.954248,41.979405
8,2^-7,0.947712,42.535487


## EOS

In [49]:
eos_exp = QuantileSVRExperiment(
    X=X_eos,
    y=y_eos,
    satellite="EOS-04",
    print_stats=False,
    type="uncensored" # or "uncensored"
)

all_results = []

for i, gamma in enumerate(GAMMA_VALUES):
    print(f"\n[RUN {i+1}/{len(GAMMA_VALUES)}]")
    print(f"Testing Gamma = {gamma} (2^{np.log2(gamma):.0f})")

    try:
        results = eos_exp.run_experiment(
            C_value=C_VALUE,
            gamma=gamma
        )
        all_results.append(results)
        # print(f"Run {i+1} complete. Test PICP: {results['test']['PICP']:.4f}, Test MPIW: {results['test']['MPIW']:.4f}")

    except Exception as e:
        print(f"ERROR: Run failed for Gamma = {gamma}")
        print(f"Details: {e}")
        all_results.append({
            "params": {"C": C_VALUE, "gamma": gamma},
            "val": {"PICP": None, "MPIW": None},
            "test": {"PICP": None, "MPIW": None},
            "error": str(e)
        })


[RUN 1/31]
Testing Gamma = 3.0517578125e-05 (2^-15)

--- Running Quantile SVR for EOS-04 ---
Params: C=64_gamma=3.0517578125e-05
Training lower quantile (0.025) model...
Done in 80.56s
Training upper quantile (0.975) model...
Done in 65.41s
Generating predictions...
Evaluating and plotting results...
Plot saved to results_qsvr/qsvr_pi_estimation_uncensored/plots/EOS-04_C=64_gamma=3.0517578125e-05.png
Results for C=64_gamma=3.0517578125e-05:
{
    "params": {
        "C": 64,
        "gamma": 3.0517578125e-05,
        "q_lower": 0.025,
        "q_upper": 0.975
    },
    "val": {
        "PICP": 0.9803921568627451,
        "MPIW": 43.45564506920671
    },
    "test": {
        "PICP": 0.9804878048780488,
        "MPIW": 43.45564723805305
    }
}

[RUN 2/31]
Testing Gamma = 6.103515625e-05 (2^-14)

--- Running Quantile SVR for EOS-04 ---
Params: C=64_gamma=6.103515625e-05
Training lower quantile (0.025) model...
Done in 78.49s
Training upper quantile (0.975) model...
Done in 61.43s
Gene

In [50]:
results_df = pd.json_normalize(all_results)
results_df.columns = results_df.columns.str.replace('params.', 'param_')

try:
    # Calculate the exponent (e.g., 11 from 2048)
    gamma_exponents = np.log2(results_df['param_gamma']).astype(int)
    # Create the new string column (e.g., "2^11")
    results_df['param_gamma_str'] = '2^' + gamma_exponents.astype(str)
except Exception as e:
    print(f"Could not create 'param_gamma_str': {e}")
    results_df['param_gamma_str'] = results_df['param_gamma'] # Fallback
# --- END NEW CODE ---

# Save the full summary to a CSV
summary_filename = eos_exp.results_path / f"eos-04-tuning_summary_C={C_VALUE}.csv"
results_df.to_csv(summary_filename, index=False)
print(f"\nFull tuning summary saved to:\n{summary_filename}")

# --- Analysis ---
acceptable_coverage_df = results_df[
    (results_df['test.PICP'] > 0.90) & (results_df['test.PICP'] <= 0.97)
].copy()

if not acceptable_coverage_df.empty:
    acceptable_coverage_df = acceptable_coverage_df.sort_values(by="test.MPIW", ascending=True)
    print("\n--- Top Models with 90-97% Test PICP (sorted by width) ---")
    print(acceptable_coverage_df[['param_gamma_str', 'test.PICP', 'test.MPIW']].head())
else:
    print("\nNo models achieved acceptable test PICP (90-97%).")

print("\n--- Top Models (sorted by Test PICP) ---")
results_df.sort_values(by="test.PICP", ascending=False)[['param_gamma_str', 'test.PICP', 'test.MPIW']].head(5)


Full tuning summary saved to:
results_qsvr/qsvr_pi_estimation_uncensored/eos-04-tuning_summary_C=64.csv

--- Top Models with 90-97% Test PICP (sorted by width) ---
   param_gamma_str  test.PICP  test.MPIW
22             2^7   0.946341  37.245889
21             2^6   0.951220  38.829103
20             2^5   0.956098  39.206343
19             2^4   0.946341  39.426878
18             2^3   0.951220  39.794027

--- Top Models (sorted by Test PICP) ---


,param_gamma_str,test.PICP,test.MPIW
0,2^-15,0.980488,43.455647
1,2^-14,0.980488,43.444628
2,2^-13,0.980488,43.422591
3,2^-12,0.980488,43.385281
4,2^-11,0.980488,43.370563


In [51]:
results_df.sort_values(by="test.PICP", ascending=False)[['param_gamma_str', 'test.PICP', 'test.MPIW']]

,param_gamma_str,test.PICP,test.MPIW
0,2^-15,0.980488,43.455647
1,2^-14,0.980488,43.444628
2,2^-13,0.980488,43.422591
3,2^-12,0.980488,43.385281
4,2^-11,0.980488,43.370563
5,2^-10,0.980488,43.341137
6,2^-9,0.980488,43.140293
7,2^-8,0.975610,42.866527
8,2^-7,0.975610,42.688087
9,2^-6,0.965854,42.464121
